In [ ]:
!pip install aquacrop==2.2
!pip install statsmodels==0.13.2
!pip install dfply
!pip install seaborn
!pip install pyshp
!pip install geopandas

In [ ]:
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
from aquacrop.utils import prepare_weather, get_filepath
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement
#from aquacrop.entities import IrrigationManagement
from os import chdir, getcwd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import csv
import pickle 
#from dfply import *


import os 
import datetime
import shapefile as shp
import pandas as pd
import geopandas as gpd
from shapely import geometry
import math
from os import listdir
from os.path import isfile, join
import glob

In [ ]:
# function to merge polygons by crop type, irrigation management and year
def geodf(crops_county):
    """ """
    crops_gdf = gpd.GeoDataFrame(crops_county)
    crops_gdf = crops_gdf.dissolve(['cropName', 'irrig_management', 'Year'])

    crops_gdf.reset_index(inplace=True) # reset index

    #create new column (id like) that identifies crops and irrig_mant
    crops_gdf['crop_mn'] = crops_gdf['cropName'] + ' ' + crops_gdf['irrig_management']
    crops_gdf['crop_mnyear'] = crops_gdf['crop_mn'] + ' ' + crops_gdf['name'].astype(str)
    
    # create code for crop mngt
    crops_gdf['crop_mn_code'] = np.where(
         crops_gdf['crop_mn'] == 'Corn irrigated', '1',
         np.where(
              crops_gdf['crop_mn'] == 'Corn rainfed', '2',
              np.where(
                   crops_gdf['crop_mn'] == 'Sorghum irrigated', '3',
                   np.where(
                        crops_gdf['crop_mn'] == 'Sorghum rainfed', '4',
                        np.where(
                             crops_gdf['crop_mn'] == 'Soybeans irrigated', '5',
                             np.where(
                                  crops_gdf['crop_mn'] == 'Soybeans rainfed', '6',
                                  np.where(
                                       crops_gdf['crop_mn'] == 'Winter Wheat irrigated', '7', '8'
                                       #np.where(
                                            #logan_gdf['crop_mn'] == 'Winter Wheat irrigated', '8',
                                            #'unknown')
                                       )
                                  )
                             )
                        )
                   )
              )
         )
    
    crops_gdf['crop_mn_codeyear'] = crops_gdf['crop_mn_code'] + '_' + crops_gdf['name'].astype(str)

    return crops_gdf

## Changes that still need to be made
- correct the file import

In [ ]:
# GMD4 fields
fields = gpd.read_file(wd + '/data/agricLand/Property Lines/Kansas/gmd4/ModelExtent_GMD4model_BWilson_v1_20230111.shp')

# KS counties
# https://catalog.data.gov/dataset/tiger-line-shapefile-2019-state-kansas-current-county-subdivision-state-based
# site had wrong coordinates

# https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name&refine.state_name=Kansas

county_boundary = gpd.read_file(wd + '/data/agricLand/countyBoundaries/Kansas/USCountyBoundaries_v1_WNdlovu_20221220.shp')


In [ ]:
# filter for countys in GMD-4
gmd4 = ['Cheyenne', 'Rawlins', 'Decatur', 'Sherman', 'Gove', 'Thomas', 'Sheridan', 'Graham', 'Wallace', 'Logan'] # list of gmd4 coundties
county_boundary = county_boundary[county_boundary['name'].isin(gmd4)] # filter for counties inside gmd4 list

# county_boundary and fields layers have same projection
fields = fields.to_crs({'init': 'epsg:4326'})
county_boundary = county_boundary.to_crs({'init': 'epsg:4326'})

# Execute spatial join
fields_county = county_boundary.sjoin(fields, how="inner", predicate='intersects') # has 93390 obs vs fields with 176407 obs
dup_fields = fields_county[fields_county['UID'].duplicated(keep= False)] # fields that appear in 2 counties

fields_county = fields_county.drop_duplicates(subset=['UID']) # dropped dupilcated observations based on UID
fields_county = fields_county[['UID', 'name', 'geometry']] # select useful variables

In [ ]:
# crop data data from 2006 - 2020
#crops_irrigation = pd.read_csv(wd + '/data/water/Kansas/IrrigationDepth_GMD4_WNdlovu_v1_20230123.csv')
#crops_field = pd.read_csv(wd + '/data/agricLand/landUse_landCover/FieldsAttributesAroundSD6KS_LandCover_AnnualCDL.csv')

# merge irrigation status files
landUse_path = wd + '/data/agricLand/landUse_landCover/GMD4_OpenET-data-cdl_tables_lema' # landouse folder path
landUse_files = [f for f in listdir(landUse_path) if isfile(join(landUse_path, f))] # read files from folder
landUse_dfs_list = []  # List to store the dataframes

for file in landUse_files: # read files and save them a list of dataframes
    file_path = os.path.join(landUse_path, file)
    df = pd.read_csv(file_path) 
    landUse_dfs_list.append(df)

landUse_df = pd.concat(landUse_dfs_list, ignore_index=True) # merge dataframes in list 
landUse_df = landUse_df.drop(['system:index', 'status', '.geo'], axis=1) #drop system index variable

# replace all nan with 0 except for year and masterid cols
cols = ['Year', 'masterid']

landUse_df.loc[:, ~landUse_df.columns.isin(cols)] = landUse_df.loc[:, ~landUse_df.columns.isin(cols)].fillna(0)

# make year and masterid indexes
landUse_df = landUse_df.set_index(['Year', 'masterid'])
landUse_df['totalCover'] = landUse_df.sum(axis=1, numeric_only=True) #calculate (tatal land used) rowwise sum for all crops 

landUse_df = landUse_df[['1', '4', '5', '24', 'totalCover']] # select only the crops of interest "Corn", "Sorghum", "Soybeans", "Winter Wheat"]

# calculate % land cover
landUse_df['1'] = landUse_df['1']/landUse_df['totalCover']
landUse_df['4'] = landUse_df['4']/landUse_df['totalCover']
landUse_df['5'] = landUse_df['5']/landUse_df['totalCover']
landUse_df['24'] = landUse_df['24']/landUse_df['totalCover']

landUse_df = landUse_df.drop(['totalCover'], axis=1) # drop totalCover 
landUse_df = landUse_df.loc[~(landUse_df==0).all(axis=1)] # remove fields with 0% cover for all four crops
landUse_df['CropCode'] = landUse_df[['1','4', '5', '24']].idxmax(axis=1) # crop with highest cover crop code
landUse_df = landUse_df.drop(['1','4', '5', '24'], axis=1) # drop % cover
landUse_df = landUse_df.reset_index()
landUse_df = landUse_df.rename(columns={'masterid': 'UID'}) # rename columns
landUse_df = landUse_df[landUse_df['UID'] != 0] # remove place holder row with uid of 0

landUse_df['CropCode'] = landUse_df['CropCode'].astype(int)

In [ ]:
crops_field = landUse_df.merge(crops, on='CropCode', how='left')

# one df with field and crop data
crops_irrig = crops_field.merge(irrig_status, on=['UID', 'Year'], how='inner') # create a dataframe which shows the field UID, irrigation status and the crop grown 
crops_irrig = crops_irrig.merge(fields_county, on=['UID'], how='inner') # merge to get df with the county

In [ ]:
# county level yield for different crops (corn, wheat, sorghum and soybeans)
corn_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/corn.csv')
corn_gmd4['Irrig_status'] = np.where(corn_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
corn_gmd4 = corn_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

wheat_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/winter_wheat.csv')
wheat_gmd4['Irrig_status'] = np.where(wheat_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
wheat_gmd4 = wheat_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

soybeans_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/soybeans.csv')
soybeans_gmd4['Irrig_status'] = np.where(soybeans_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
soybeans_gmd4 = soybeans_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

sorghum_gmd4 = pd.read_csv(wd + '/data/agricLand/yield/Kansas/sorghum.csv')
sorghum_gmd4['Irrig_status'] = np.where(sorghum_gmd4['Data Item'].str.contains("NON-IRRIGATED"), 'rainfed', 'irrigated')
sorghum_gmd4 = sorghum_gmd4[["Year", "State", "County", "Commodity", "Data Item", "Irrig_status", "Value"]]

# irrigation status
irrig_status = pd.read_csv(wd + '/data/agricLand/irrigationStatus/Kansas/AIM_HPA_GMD4_2000_2020_Deines_etal_RSE_v01_samsPolygons.csv')
irrig_status['0'] = irrig_status['0'].fillna(0) # replace nan with 0
irrig_status['1'] = irrig_status['1'].fillna(0) # replace nan with 0
irrig_status['IrrigatedPrc'] = 1 - (irrig_status['0'] / (irrig_status['0'] + irrig_status['1']))
irrig_status = irrig_status.assign(irrig_management = np.where(irrig_status['IrrigatedPrc'] <= 0.5, 'rainfed', 'irrigated')) # variable with irrigation status
irrig_status = irrig_status[['UID', 'year', 'IrrigatedPrc', 'irrig_management']]
irrig_status = irrig_status.rename(columns={'year': 'Year'})
#y = irrig_status.tail(5)

In [ ]:
# Create lists of crop codes, crop names, and crop groups to match them up in a single dataframe
CropCode = [1, 4, 5, 24]

crop_name = ["Corn", 
             "Sorghum", 
             "Soybeans",  
             "Winter Wheat"]


crops = pd.DataFrame(zip(CropCode, crop_name), columns=['CropCode', 'cropName'])

# new column with crop name int he crops_fields (full df)
crops_field = landUse_df.merge(crops, on='CropCode', how='left')

# one df with field and crop data
crops_irrig = crops_field.merge(irrig_status, on=['UID', 'Year'], how='inner') # create a dataframe which shows the field UID, irrigation status and the crop grown 
crops_irrig = crops_irrig.merge(fields_county, on=['UID'], how='inner') # merge to get df with the county

In [ ]:
# spilt by the county and save as separate df
#'Cheyenne', 'Rawlins', 'Decatur', 'Sherman', 'Gove', 'Thomas', 'Sheridan', 'Graham', 'Wallace', 'Logan'
crops_cheyenne = crops_irrig[crops_irrig['name'] == 'Cheyenne']
crops_rawlins = crops_irrig[crops_irrig['name'] == 'Rawlins']
crops_decatur = crops_irrig[crops_irrig['name'] == 'Decatur']
crops_sherman = crops_irrig[crops_irrig['name'] == 'Sherman']
crops_gove = crops_irrig[crops_irrig['name'] == 'Gove']
crops_thomas = crops_irrig[crops_irrig['name'] == 'Thomas']
crops_sheridan = crops_irrig[crops_irrig['name'] == 'Sheridan']
crops_graham = crops_irrig[crops_irrig['name'] == 'Graham']
crops_wallace = crops_irrig[crops_irrig['name'] == 'Wallace']
crops_logan = crops_irrig[crops_irrig['name'] == 'Logan']



# create list of the dfs 
crops_county_gdf = [crops_cheyenne, crops_rawlins, crops_decatur, crops_sherman, 
                    crops_gove, crops_thomas, crops_sheridan, crops_graham, crops_wallace, crops_logan]

In [ ]:
crops_county_lst = list(map(geodf, crops_county_gdf)) # list of county dfs

#merge and separate by year
crops_county = pd.concat(crops_county_lst)

years = crops_county['Year'].unique().astype(int)  # list of years
output_dir = r'./data//agricLand/Crop County Geometries'  # Specify the output directory

for year in years:
    # Filter the dataset for the specified year
    filtered_df = crops_county[crops_county['Year'] == year]

    # Define the output filename for the filtered dataset
    final_df = f'CropCountyGeometry_GMD4_{year}_WNdlovu_v1_20230609.csv'

    # Save the filtered dataset as a CSV file
    output_path = f'{output_dir}/{final_df}'
    filtered_df.to_csv(output_path, index=False)

# Next Steps

This .csv files are then used to download soil (polaris), met (gridMET) and leaf area index data from gee. 